# SAA based agents

> Agents based on Sample Average Approximation (SAA) or weighted Sample Average Approximation (wSAA)

In [ ]:
#| default_exp agents.newsvendor.saa

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import logging

from abc import ABC, abstractmethod
from typing import Union, Optional, List
import numpy as np
import joblib
import os

from ddopai.envs.base import BaseEnvironment
from ddopai.agents.base import BaseAgent
from ddopai.utils import MDPInfo
from ddopai.agents.obsprocessors import FlattenTimeDimNumpy

from sklearn.ensemble import RandomForestRegressor
from sklearn.utils.validation import check_array

In [ ]:
#| export
class BaseSAAagent(BaseAgent):

    """
    Base class for Sample Average Approximation Agents, implementing the main method
    to find the quntile of some (weighted) empirical distribution.
    """

    def __init__(self,
                 environment_info: MDPInfo,
                 obsprocessors: Optional[List[object]] = None,
                 agent_name: str | None = None,
                 ):

        super().__init__(environment_info = environment_info, obsprocessors = obsprocessors, agent_name = agent_name)

        # check if FlattenTimeDimNumpy in obsprocessors
        if not any(isinstance(p, FlattenTimeDimNumpy) for p in self.obsprocessors):
            self.add_obsprocessor(FlattenTimeDimNumpy(allow_2d=True))

    def find_weighted_quantiles(self, weights, weightPosIndices, sl, y):
        
        """
        Find the weighted quantile of a range of data y. 
        It assumes that all arrays are of shape (n_samples, n_outputs). Note that it
        has not been tested for n_outputs > 1.
        """

        # test shapes have lenght 2 with error
        assert len(y.shape) == 2, "y should be of shape (n_samples, n_outputs)"

        n_outputs = y.shape[1]

        yWeightPos = y[weightPosIndices]

        if self.print:
            print(yWeightPos)
        
        q = []

        if len(weights.shape) == 1:
            weights = weights.reshape(-1, 1)
        
        for i in range(n_outputs):
            
            indicesYSort = np.argsort(yWeightPos[:, i])
            
            ySorted = yWeightPos[indicesYSort, i]
            
            distributionFunction = np.cumsum(weights[indicesYSort, i])

            decisionIndex = np.where(distributionFunction >= sl)[0][0]
            
            q.append(ySorted[decisionIndex])

        q = np.array(q)
        
        return q
    
    def _validate_X_predict(self, X):
        
        """Validate X data before prediction"""

        X = check_array(X)

        n_features = X.shape[1]
        if self.n_features_ != n_features:
            raise ValueError("Number of features of the model must match the input. "
                             "Model n_features is %s and input n_features is %s "
                             % (self.n_features_, n_features))
        return X

In [ ]:
show_doc(BaseSAAagent, title_level=2)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L25){target="_blank" style="float:right; font-size:smaller"}

## BaseSAAagent

>      BaseSAAagent (environment_info:ddopai.utils.MDPInfo,
>                    obsprocessors:Optional[List[object]]=None,
>                    agent_name:str|None=None)

*Base class for Sample Average Approximation Agents, implementing the main method
to find the quntile of some (weighted) empirical distribution.*

In [ ]:
show_doc(BaseSAAagent._validate_X_predict)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L84){target="_blank" style="float:right; font-size:smaller"}

### BaseSAAagent._validate_X_predict

>      BaseSAAagent._validate_X_predict (X)

*Validate X data before prediction*

In [ ]:
show_doc(BaseSAAagent.find_weighted_quantiles)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L45){target="_blank" style="float:right; font-size:smaller"}

### BaseSAAagent.find_weighted_quantiles

>      BaseSAAagent.find_weighted_quantiles (weights, weightPosIndices, sl, y)

*Find the weighted quantile of a range of data y. 
It assumes that all arrays are of shape (n_samples, n_outputs). Note that it
has not been tested for n_outputs > 1.*

In [ ]:
#| export
class NewsvendorSAAagent(BaseSAAagent):

    """
    Newsvendor agent that uses Sample Average Approximation to find the quantile of the empirical distribution

    """

    def __init__(self,
                environment_info: MDPInfo,
                cu: float | np.ndarray, # underage cost
                co: float | np.ndarray, # overage cost
                obsprocessors: list[object] | None = None,
                agent_name: str = "SAA",
                ):

            # if float, convert to array
            cu = self.convert_to_numpy_array(cu)
            co = self.convert_to_numpy_array(co)

            self.sl = cu / (cu + co)
            self.fitted = False

            super().__init__(environment_info = environment_info, obsprocessors = obsprocessors, agent_name = agent_name)

    def fit(self,
            X: np.ndarray, # features will be ignored
            Y: np.ndarray) -> None:

        """

        Fit the agent to the data. The agent will find the quantile of the empirical distribution of the data.

        """

        # # potential line:
        # X, y = self._validate_data(X, y, multi_output=True)

        weights = np.ones(Y.shape)/Y.shape[0]
        weightPosIndices = np.arange(Y.shape[0])
        
        self.quantiles = self.find_weighted_quantiles(weights, weightPosIndices, self.sl, Y)

        self.fitted = True

    def draw_action_(self, 
                    observation: np.ndarray) -> np.ndarray: #
        """

        Draw an action from the quantile of the empirical distribution.

        """


        if self.fitted == False:
            return np.array([0.0])

        return self.quantiles


    def save(self,
                path: str, # The directory where the file will be saved.
                overwrite: bool=True): # Allow overwriting; if False, a FileExistsError will be raised if the file exists.
        
        """
        Save the quantiles to a file in the specified directory.

        """

        if not self.fitted:
            raise ValueError("Agent has not been fitted yet")

        os.makedirs(path, exist_ok=True)
        
        full_path = os.path.join(path, "saa_quantiles.npy")
        
        if os.path.exists(full_path):
            if not overwrite:
                raise FileExistsError(f"The file {full_path} already exists and will not be overwritten.")
            else:
                logging.warning(f"Overwriting file {full_path}")
                
        np.save(full_path, self.quantiles)

    def load(self, path: str): # Only the path to the folder is needed, not the file itself

        """
        Load the quantiles from a file.
        

        """

        full_path = os.path.join(path, "saa_quantiles.npy")
        
        if not os.path.exists(full_path):
            raise FileNotFoundError(f"The file {full_path} does not exist.")
        
        try:
            self.quantiles = np.load(full_path)
            self.fitted = True  # Assuming that loading the quantiles means the agent is now 'fitted'
            logging.info(f"Quantiles loaded successfully from {full_path}")
        except Exception as e:
            raise ValueError(f"An error occurred while loading the file: {e}")

In [ ]:
show_doc(NewsvendorSAAagent, title_level=2)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L98){target="_blank" style="float:right; font-size:smaller"}

## NewsvendorSAAagent

>      NewsvendorSAAagent (environment_info:ddopai.utils.MDPInfo,
>                          cu:float|numpy.ndarray, co:float|numpy.ndarray,
>                          obsprocessors:list[object]|None=None,
>                          agent_name:str='SAA')

*Newsvendor agent that uses Sample Average Approximation to find the quantile of the empirical distribution*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| environment_info | MDPInfo |  |  |
| cu | float \| numpy.ndarray |  | underage cost |
| co | float \| numpy.ndarray |  | overage cost |
| obsprocessors | list[object] \| None | None |  |
| agent_name | str | SAA |  |

#### Further information:

References:

    .. [1] Levi, Retsef, Georgia Perakis, and Joline Uichanco. "The data-driven newsvendor problem: new bounds and insights."
           Operations Research 63.6 (2015): 1294-1306.

In [ ]:
show_doc(NewsvendorSAAagent.fit)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L123){target="_blank" style="float:right; font-size:smaller"}

### NewsvendorSAAagent.fit

>      NewsvendorSAAagent.fit (X:numpy.ndarray, Y:numpy.ndarray)

*Fit the agent to the data. The agent will find the quantile of the empirical distribution of the data.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | ndarray | features will be ignored |
| Y | ndarray |  |
| **Returns** | **None** |  |

In [ ]:
show_doc(NewsvendorSAAagent.draw_action_)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L143){target="_blank" style="float:right; font-size:smaller"}

### NewsvendorSAAagent.draw_action_

>      NewsvendorSAAagent.draw_action_ (observation:numpy.ndarray)

*Draw an action from the quantile of the empirical distribution.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| observation | ndarray |  |
| **Returns** | **ndarray** |  |

In [ ]:
show_doc(NewsvendorSAAagent.save)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L158){target="_blank" style="float:right; font-size:smaller"}

### NewsvendorSAAagent.save

>      NewsvendorSAAagent.save (path:str, overwrite:bool=True)

*Save the quantiles to a file in the specified directory.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| path | str |  | The directory where the file will be saved. |
| overwrite | bool | True | Allow overwriting; if False, a FileExistsError will be raised if the file exists. |

In [ ]:
show_doc(NewsvendorSAAagent.load)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L182){target="_blank" style="float:right; font-size:smaller"}

### NewsvendorSAAagent.load

>      NewsvendorSAAagent.load (path:str)

*Load the quantiles from a file.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| path | str | Only the path to the folder is needed, not the file itself |

In [ ]:
#| export
class BasewSAAagent(BaseSAAagent):


    """
    Base class for weighted Sample Average Approximation (wSAA) Agents

    """

    def __init__(self,
                environment_info: MDPInfo,
                cu: float | np.ndarray,
                co: float | np.ndarray,
                obsprocessors: list[object] | None = None,
                agent_name: str = "wSAA",
                ):  #


        # if float, convert to array
        self.cu = np.array([cu]) if isinstance(cu, float) else cu
        self.co = np.array([co]) if isinstance(co, float) else co

        cu = self.convert_to_numpy_array(cu)
        co = self.convert_to_numpy_array(co)

        self.sl = cu / (cu + co)
        
        self.fitted = False

        super().__init__(environment_info = environment_info, obsprocessors = obsprocessors, agent_name = agent_name)

    def fit(self,
            X: np.ndarray,
            Y: np.ndarray): #

        """

        Fit the agent to the data. The function will call _get_fitted_model which will
        train a machine learning model to determine the sample weightes (e.g., kNN, DT, RF).

        """
        
        # # potential line:
        # X, y = self._validate_data(X, y, multi_output=True)

        # get FlattenTimeDimNumpy obsprocessor
        flatten_obsprocessor = [p for p in self.obsprocessors if isinstance(p, FlattenTimeDimNumpy)][0]

        X = flatten_obsprocessor(X)

        if len(Y.shape) == 2 and Y.shape[1] == 1:
            Y = Y.flatten() 

        self._get_fitted_model(X, Y)

        if Y.ndim == 1:
            Y = np.reshape(Y, (-1, 1))

        # Training data
        self.Y_ = Y
        self.X_ = X
        self.n_samples_ = Y.shape[0]

        # Determine output settings
        self.n_outputs_ = Y.shape[1]
        self.n_features_ = X.shape[1]

        self.fitted=True

    def draw_action_(self, 
                    observation: np.ndarray) -> np.ndarray: # 

        """

        Draw an action based on the fitted model (see predict method)

        """

        if self.fitted == False:
            return np.array([0.0])
        
        return self.predict(observation)
    
    @abstractmethod
    def _get_fitted_model(self, X, y):
        """Initialise the underlying model - depending on the underlying machine learning model"""

    @abstractmethod
    def _calc_weights(self, sample):
        """Calculate the sample weights - depending on the underlying machine learning model"""

    def predict(self, 
                X: np.ndarray
    ) -> np.ndarray: #
        """Predict value for X by finding the quantiles of the empirical distribution based
        on the sample weights predicted by the underlying machine learning model.
        """

        X = self._validate_X_predict(X)  

        if self.print:
            print("X: ", X)

        weightsDataList = [self._calc_weights(row) for row in X]

        if self.print:
            print("weightsDataList: ", weightsDataList)

        pred = [self.find_weighted_quantiles(weights, weightPosIndices, self.sl, self.Y_) 
                for weights, weightPosIndices in weightsDataList]


        pred = np.array(pred)   

        if self.print:
            print("Predicted quantiles: ", pred)

        return pred

    def save(self,
                path: str, # The directory where the file will be saved.
                overwrite: bool=True): # Allow overwriting; if False, a FileExistsError will be raised if the file exists.
        """
        Save the scikit-learn model to a file in the specified directory.

        """

        if not self.fitted:
            raise ValueError("Agent has not been fitted yet")

        if not hasattr(self, 'model_') or self.model_ is None:
            raise ValueError("Agent has no model to save.")

        # Create directory if it does not exist
        os.makedirs(path, exist_ok=True)
        
        # Construct the file path using os.path.join for better cross-platform compatibility
        full_path = os.path.join(path, "model.joblib")
        
        if os.path.exists(full_path):
            if not overwrite:
                raise FileExistsError(f"The file {full_path} already exists and will not be overwritten.")
            else:
                logging.warning(f"Overwriting file {full_path}")
        
        # Save the model using joblib
        joblib.dump(self.model_, full_path)

    def load(self, path: str): # Only the path to the folder is needed, not the file itself
        """
        Load the scikit-learn model from a file.

        """
        
        # Construct the file path
        full_path = os.path.join(path, "model.joblib")
        
        if not os.path.exists(full_path):
            raise FileNotFoundError(f"The file {full_path} does not exist.")
        
        try:
            # Load the model using joblib
            self.model_ = joblib.load(full_path)
            self.fitted = True  # Assuming that loading the model means the agent is now 'fitted'
            logging.info(f"Model loaded successfully from {full_path}")
        except Exception as e:
            raise ValueError(f"An error occurred while loading the model: {e}")

In [ ]:
show_doc(BasewSAAagent, title_level=2)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L203){target="_blank" style="float:right; font-size:smaller"}

## BasewSAAagent

>      BasewSAAagent (environment_info:ddopai.utils.MDPInfo,
>                     cu:float|numpy.ndarray, co:float|numpy.ndarray,
>                     obsprocessors:list[object]|None=None,
>                     agent_name:str='wSAA')

*Base class for weighted Sample Average Approximation (wSAA) Agents*

In [ ]:
show_doc(BasewSAAagent.fit)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L234){target="_blank" style="float:right; font-size:smaller"}

### BasewSAAagent.fit

>      BasewSAAagent.fit (X:numpy.ndarray, Y:numpy.ndarray)

*Fit the agent to the data. The function will call _get_fitted_model which will
train a machine learning model to determine the sample weightes (e.g., kNN, DT, RF).*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | ndarray |  |
| Y | ndarray |  |

In [ ]:
show_doc(BasewSAAagent.draw_action)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/base.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### BaseAgent.draw_action

>      BaseAgent.draw_action (observation:numpy.ndarray)

*Main interfrace to the environemnt. Applies preprocessors to the observation.
Internal logic of the agent to be implemented in draw_action_ method.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| observation | ndarray |  |
| **Returns** | **ndarray** |  |

In [ ]:
show_doc(BasewSAAagent._get_fitted_model)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L287){target="_blank" style="float:right; font-size:smaller"}

### BasewSAAagent._get_fitted_model

>      BasewSAAagent._get_fitted_model (X, y)

*Initialise the underlying model - depending on the underlying machine learning model*

In [ ]:
show_doc(BasewSAAagent._calc_weights)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L291){target="_blank" style="float:right; font-size:smaller"}

### BasewSAAagent._calc_weights

>      BasewSAAagent._calc_weights (sample)

*Calculate the sample weights - depending on the underlying machine learning model*

In [ ]:
show_doc(BasewSAAagent.predict)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L294){target="_blank" style="float:right; font-size:smaller"}

### BasewSAAagent.predict

>      BasewSAAagent.predict (X:numpy.ndarray)

*Predict value for X by finding the quantiles of the empirical distribution based
on the sample weights predicted by the underlying machine learning model.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | ndarray |  |
| **Returns** | **ndarray** |  |

In [ ]:
show_doc(BasewSAAagent.save)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L322){target="_blank" style="float:right; font-size:smaller"}

### BasewSAAagent.save

>      BasewSAAagent.save (path:str, overwrite:bool=True)

*Save the scikit-learn model to a file in the specified directory.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| path | str |  | The directory where the file will be saved. |
| overwrite | bool | True | Allow overwriting; if False, a FileExistsError will be raised if the file exists. |

In [ ]:
show_doc(BasewSAAagent.load)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L351){target="_blank" style="float:right; font-size:smaller"}

### BasewSAAagent.load

>      BasewSAAagent.load (path:str)

*Load the scikit-learn model from a file.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| path | str | Only the path to the folder is needed, not the file itself |

In [ ]:
#| export
class NewsvendorRFwSAAagent(BasewSAAagent):

    """

    Newsvendor agent that uses weighted Sample Average Approximation based on Random Forest

    """

    def __init__(self,
                environment_info: MDPInfo,
                cu: float | np.ndarray, # underage cost
                co: float | np.ndarray, # overage cost
                obsprocessors: list[object] | None = None, # List of obsprocessors to apply to the observation
                n_estimators: int = 100,# The number of trees in the forest.
                criterion: str = "squared_error", # Function to measure the quality of a split.
                max_depth: int | None = None, # Maximum depth of the tree; None means unlimited.
                min_samples_split: int = 2, # Minimum samples required to split a node.
                min_samples_leaf: int = 1, # Minimum samples required to be at a leaf node.
                min_weight_fraction_leaf: float = 0.0, # Minimum weighted fraction of the total weights at a leaf node.
                max_features: int | float | str | None = 1.0, # Number of features to consider when looking for the best split.
                max_leaf_nodes: int | None = None, # Maximum number of leaf nodes; None means unlimited.
                min_impurity_decrease: float = 0.0, # Minimum impurity decrease required to split a node.
                bootstrap: bool = True, # Whether to use bootstrap samples when building trees.
                oob_score: bool = False, # Whether to use out-of-bag samples to estimate R^2 on unseen data.
                n_jobs: int | None = None, # Number of jobs to run in parallel; None means 1.
                random_state: int | np.random.RandomState | None = None, # Controls randomness for bootstrapping and feature sampling.
                verbose: int = 0, # Controls the verbosity when fitting and predicting.
                warm_start: bool = False, # If True, reuse solution from previous fit and add more estimators.
                ccp_alpha: float = 0.0, # Complexity parameter for Minimal Cost-Complexity Pruning.
                max_samples: int | float | None = None, # Number of samples to draw when bootstrap is True.
                monotonic_cst: np.ndarray | None = None, # Monotonic constraints for features.
                agent_name: str = "wSAA", # Default wSAA, change if it is needed to differentiate among different ML models
                ):
        self.criterion = criterion
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_weight_fraction_leaf = min_weight_fraction_leaf
        self.max_features = max_features
        self.max_leaf_nodes = max_leaf_nodes
        self.min_impurity_decrease = min_impurity_decrease
        self.bootstrap = bootstrap
        self.oob_score = oob_score
        self.n_jobs = n_jobs
        self.random_state = random_state
        self.verbose = verbose
        self.warm_start = warm_start
        self.ccp_alpha = ccp_alpha
        self.max_samples = max_samples
        self.monotonic_cst = monotonic_cst
        self.weight_function = "w1"

        super().__init__(environment_info = environment_info, cu = cu, co = co, obsprocessors = obsprocessors, agent_name = agent_name)

    def _get_fitted_model(self,
                            X: np.ndarray,
                            Y: np.ndarray): #

        """

        Fit the underlying machine learning model using all X and Y data in the train set.

        """

        model = RandomForestRegressor(
            criterion=self.criterion,
            n_estimators=self.n_estimators,
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split,
            min_samples_leaf=self.min_samples_leaf,
            min_weight_fraction_leaf=self.min_weight_fraction_leaf,
            max_features=self.max_features,
            max_leaf_nodes=self.max_leaf_nodes,
            min_impurity_decrease=self.min_impurity_decrease,
            bootstrap=self.bootstrap,
            oob_score=self.oob_score,
            n_jobs=self.n_jobs,
            random_state=self.random_state,
            verbose=self.verbose,
            warm_start=self.warm_start,
            ccp_alpha=self.ccp_alpha,
            max_samples=self.max_samples,
            monotonic_cst = self.monotonic_cst
        )

        self.model_ = model.fit(X, Y)
        self.train_leaf_indices_ = model.apply(X)

    def _calc_weights(self, sample: np.ndarray) -> tuple[np.ndarray, np.ndarray]: #

        """
        Calculate the sample weights based on the Random Forest model.

        """

        sample_leaf_indices = self.model_.apply([sample])
        if self.weight_function == "w1":
            n = np.sum(sample_leaf_indices == self.train_leaf_indices_, axis=0)
            treeWeights = (sample_leaf_indices == self.train_leaf_indices_) / n
            weights = np.sum(treeWeights, axis=1) / self.n_estimators
        else:
            n = np.sum(sample_leaf_indices == self.train_leaf_indices_)
            treeWeights = (sample_leaf_indices == self.train_leaf_indices_) / n
            weights = np.sum(treeWeights, axis=1)
        
        weightPosIndex = np.where(weights > 0)[0]
        weightsPos = weights[weightPosIndex]

        return (weightsPos, weightPosIndex)

In [ ]:
show_doc(NewsvendorRFwSAAagent, title_level=2)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L372){target="_blank" style="float:right; font-size:smaller"}

## NewsvendorRFwSAAagent

>      NewsvendorRFwSAAagent (environment_info:ddopai.utils.MDPInfo,
>                             cu:float|numpy.ndarray, co:float|numpy.ndarray,
>                             obsprocessors:list[object]|None=None,
>                             n_estimators:int=100,
>                             criterion:str='squared_error',
>                             max_depth:int|None=None, min_samples_split:int=2,
>                             min_samples_leaf:int=1,
>                             min_weight_fraction_leaf:float=0.0,
>                             max_features:int|float|str|None=1.0,
>                             max_leaf_nodes:int|None=None,
>                             min_impurity_decrease:float=0.0,
>                             bootstrap:bool=True, oob_score:bool=False,
>                             n_jobs:int|None=None, random_state:int|numpy.rando
>                             m.mtrand.RandomState|None=None, verbose:int=0,
>                             warm_start:bool=False, ccp_alpha:float=0.0,
>                             max_samples:int|float|None=None,
>                             monotonic_cst:numpy.ndarray|None=None,
>                             agent_name:str='wSAA')

*Newsvendor agent that uses weighted Sample Average Approximation based on Random Forest*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| environment_info | MDPInfo |  |  |
| cu | float \| numpy.ndarray |  | underage cost |
| co | float \| numpy.ndarray |  | overage cost |
| obsprocessors | list[object] \| None | None | List of obsprocessors to apply to the observation |
| n_estimators | int | 100 | The number of trees in the forest. |
| criterion | str | squared_error | Function to measure the quality of a split. |
| max_depth | int \| None | None | Maximum depth of the tree; None means unlimited. |
| min_samples_split | int | 2 | Minimum samples required to split a node. |
| min_samples_leaf | int | 1 | Minimum samples required to be at a leaf node. |
| min_weight_fraction_leaf | float | 0.0 | Minimum weighted fraction of the total weights at a leaf node. |
| max_features | int \| float \| str \| None | 1.0 | Number of features to consider when looking for the best split. |
| max_leaf_nodes | int \| None | None | Maximum number of leaf nodes; None means unlimited. |
| min_impurity_decrease | float | 0.0 | Minimum impurity decrease required to split a node. |
| bootstrap | bool | True | Whether to use bootstrap samples when building trees. |
| oob_score | bool | False | Whether to use out-of-bag samples to estimate R\^2 on unseen data. |
| n_jobs | int \| None | None | Number of jobs to run in parallel; None means 1. |
| random_state | int \| numpy.random.mtrand.RandomState \| None | None | Controls randomness for bootstrapping and feature sampling. |
| verbose | int | 0 | Controls the verbosity when fitting and predicting. |
| warm_start | bool | False | If True, reuse solution from previous fit and add more estimators. |
| ccp_alpha | float | 0.0 | Complexity parameter for Minimal Cost-Complexity Pruning. |
| max_samples | int \| float \| None | None | Number of samples to draw when bootstrap is True. |
| monotonic_cst | numpy.ndarray \| None | None | Monotonic constraints for features. |
| agent_name | str | wSAA | Default wSAA, change if it is needed to differentiate among different ML models |

#### Further information:

Notes
    -----
    
The default values for the parameters controlling the size of the trees
(e.g. ``max_depth``, ``min_samples_leaf``, etc.) lead to fully grown and
unpruned trees which can potentially be very large on some data sets. To
reduce memory consumption, the complexity and size of the trees should be
controlled by setting those parameter values.
The features are always randomly permuted at each split. Therefore,
the best found split may vary, even with the same training data,
``max_features=n_features`` and ``bootstrap=False``, if the improvement
of the criterion is identical for several splits enumerated during the
search of the best split. To obtain a deterministic behaviour during
fitting, ``random_state`` has to be fixed.


References
    ----------

    .. [1] L. Breiman, "Random Forests", Machine Learning, 45(1), 5-32, 2001.

    .. [2] P. Geurts, D. Ernst., and L. Wehenkel, "Extremely randomized
           trees", Machine Learning, 63(1), 3-42, 2006.

    .. [3] Bertsimas, Dimitris, and Nathan Kallus, "From predictive to prescriptive analytics."
           arXiv preprint arXiv:1402.5481 (2014).

    .. [4] scikit-learn, RandomForestRegressor,
           <https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/ensemble/_forest.py>
           
    .. [5] Scornet, Erwan. "Random forests and kernel methods."
           IEEE Transactions on Information Theory 62.3 (2016): 1485-1500.

In [ ]:
show_doc(NewsvendorRFwSAAagent._get_fitted_model)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L428){target="_blank" style="float:right; font-size:smaller"}

### NewsvendorRFwSAAagent._get_fitted_model

>      NewsvendorRFwSAAagent._get_fitted_model (X:numpy.ndarray,
>                                               Y:numpy.ndarray)

*Fit the underlying machine learning model using all X and Y data in the train set.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | ndarray |  |
| Y | ndarray |  |

In [ ]:
show_doc(NewsvendorRFwSAAagent._calc_weights)

---

[source](https://github.com/opimwue/ddopai/blob/main/ddopai/agents/newsvendor/saa.py#L462){target="_blank" style="float:right; font-size:smaller"}

### NewsvendorRFwSAAagent._calc_weights

>      NewsvendorRFwSAAagent._calc_weights (sample:numpy.ndarray)

*Calculate the sample weights based on the Random Forest model.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| sample | ndarray |  |
| **Returns** | **tuple** |  |

Example usage:

In [ ]:
from ddopai.envs.inventory.single_period import NewsvendorEnv
from ddopai.dataloaders.tabular import XYDataLoader
from ddopai.experiments.experiment_functions import run_experiment, test_agent

In [ ]:
val_index_start = 800 #90_000
test_index_start = 900 #100_000

X = np.random.rand(1000, 2)
Y = np.random.rand(1000, 1)

dataloader = XYDataLoader(X, Y, val_index_start, test_index_start)

environment = NewsvendorEnv(
    dataloader = dataloader,
    underage_cost = 0.42857,
    overage_cost = 1.0,
    gamma = 0.999,
    horizon_train = 365,
)

agent = NewsvendorSAAagent(environment.mdp_info, cu=0.42857, co=1.0)
agent = NewsvendorRFwSAAagent(environment.mdp_info, cu=0.42857, co=1.0)

environment.test()
agent.eval()

R, J = test_agent(agent, environment)

print(R, J)

run_experiment(agent, environment, 100, run_id = "test", save_best=True) # fit agent via run_experiment function
    
environment.test()
agent.eval()

R, J = test_agent(agent, environment)

print(R, J)

-18.01888542213257 -17.142493964355882


results
-15.763567080255545 -15.022369246527656 -15.763567080255545 -15.022369246527656
-17.334785352427232 -16.554914069406784


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()